In [2]:
import gradio as gr
import pickle
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Load the trained model for a specific city
def load_model(city_name):
    # Convert city name to lowercase
    city_name_lower = city_name.lower()
    model_file_path = f"{city_name_lower}_model.pkl"
    with open(model_file_path, 'rb') as f:
        model = pickle.load(f)
    return model

# Your forecast_temperature function
def forecast_temperature(city_name, show_average_temperature=False):
    # Convert city name to lowercase
    city_name_lower = city_name.lower()
    
    # Load the model for the specified city
    model = load_model(city_name_lower)
    
    # Assuming your model has a method to get the forecast for the next 48 hours
    forecast = model.get_forecast(steps=48)  # Replace with your actual method
    
    # Convert Series to DataFrame
    forecast_table = forecast.predicted_mean.reset_index().rename(columns={'index': 'timestamp', 0: 'temperature'})
    
    # Calculate average temperature if the option is selected
    average_temperature = forecast_table['predicted_mean'].mean() if show_average_temperature else None
    max_temperature = forecast_table['predicted_mean'].max()
    min_temperature = forecast_table['predicted_mean'].min()

    # Create the output strings
    output_string = f"Forecasted Temperature for {city_name_lower} °C:\nForecasted Table:\n{forecast_table.to_string(index=False)}"

    avg_output_string = f"Average Temperature: {average_temperature:.2f} °C" if show_average_temperature else ""
    max_output_string = f"Maximum Temperature: {max_temperature:.2f} °C"
    min_output_string = f"Minimum Temperature: {min_temperature:.2f} °C"

    return output_string, avg_output_string, max_output_string, min_output_string

# Create Gradio interface with radio buttons and checkbox
city_radio = gr.Radio(
    choices=["Agartala", "Amritsar", "Bangalore", "Chennai", "Delhi", "Faridabad", "Jabalpur", "Mumbai", "Pune", "Shimla"],
    type="value",  # Corrected parameter value to "value"
    label="Select City"
)

checkbox = gr.Checkbox(
    label="Show Average Temperature"
)

iface = gr.Interface(
    fn=forecast_temperature,
    inputs=[city_radio, checkbox],
    outputs=["text", "text", "text", "text"]
)

# Launch Gradio interface
iface.launch()


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
